## **Dashboard Testleri**

Verileri "pandas" kütüphanesini kullanarak işlerken, "plotly" kütüphanesi ile işlediğimiz verileri görselleştireceğiz.

In [1]:
import pandas as pd
import plotly.express as px


### **Excel'den Veri Okumak**

Excel'den veri okumak ve işlemek için pandas kütüphanesini kullanmamız gerekecek.

In [2]:
try:
    df_goals = pd.read_excel("data.xlsx")
except FileNotFoundError:
    print("Hata: Excel dosyası bulunamadı")

### **Veri İşleme ve Agregasyon**

Pie Grafiği için Takım Bazlı Toplam Golleri Hesaplama

In [ ]:
# Takım bazlı groupby yapıldı
# Goal sütununda bulunan verileri grup içinde toplandı 
# index verisi resetlendi
df_total_goals = df_goals.groupby("Team")["Goals"].sum().reset_index()
print(df_total_goals)

# Goals olan sütun ismi Total Goals olarak günellendi
df_total_goals.rename(columns={"Goals": "Total Goals"}, inplace = True)
print(df_total_goals)

               Team  Goals
0   Adana Demirspor      4
1        Ankaragücü      4
2        Başakşehir      5
3          Beşiktaş      5
4        Fenerbahçe      7
5       Galatasaray      6
6         Kasımpaşa      4
7       Kayserispor      5
8        Pendikspor      3
9          Rizespor      3
10        Sivasspor      5
11      Trabzonspor      6
               Team  Total Goals
0   Adana Demirspor            4
1        Ankaragücü            4
2        Başakşehir            5
3          Beşiktaş            5
4        Fenerbahçe            7
5       Galatasaray            6
6         Kasımpaşa            4
7       Kayserispor            5
8        Pendikspor            3
9          Rizespor            3
10        Sivasspor            5
11      Trabzonspor            6


### **İnteraktif Hover Metni Oluşturma**

In [9]:
# Belirtilen takımın tüm oyuncularını ve gollerini satır satır yazacak.

def create_hover_text(team_name):

    # Ana veri setinden sadece ilgili takım filtrelenecek
    team_data = df_goals[df_goals["Team"] == team_name]
    text = ""

    # Filtrelenen her satır (Oyuncu) için bilgileri ekle
    for _, row in team_data.iterrows():
        text += f"{row["Player"]}: {["Row"]} gol\n"
    
    return text.strip() # Baştaki/sondaki boşlukları kaldırır




### **Oluşturulan Hover Metnini ANa DataFrame'e Ekleme**

In [10]:
df_total_goals["Hover_Text"] = df_total_goals["Team"].apply(create_hover_text)

### **Plotly ile Veri Görselleştirme**

In [15]:
fig = px.pie(
    df_total_goals,
    values="Total Goals", # Pasta dlimlerinin büyüklüğü (Toplam Gol)
    names="Team", # Dilimlerin etiketleri (Takım Adı)
    title="Türkiye Süper Ligi - 15.Hafta Takım Bazlı Gol Dağılımı (Fake Data)",
    hole=0.3 # Donut grafiği yapmak için
)

### **Custom Hover Template Ayarları**

In [ ]:
# Bu adım, interaktifliği sağlayacak
fig.update_traces(
    # "customdata" ile "Hover_Text" sütununu grafiğin her dilimine gizlice ekliyoruz.
    customdata = df_total_goals[["Hover_Text"]],

    # "hovertemplate" ile hover anında ne görünecğeini HTML ve Plotly değişkenleriyle belirliyoruz.
    hovertemlate = "<b>%{label}</b><br><br>" + # Takımın Adı (Kalın)
    "Toplam Gol: %{value}<br>" + # Takımın Toplam Golü
    "Oyuncu Detayları:<br>" +
    # %{customdata[0]} gizlediğimiz "Hover_Text" bilgisini getirir.
    # <pre> etiketi, metnin içindeki gizli satır atlamaları (\n) korumasını sağlar.
    "<pre>%{customdata[0]}</pre>" +
    "<extra>"
)